
## **Project Overview**
**The purpose of this notebook is to ingest, clean, and transform five raw source files into four cleaned, standardized datasets.**

This process includes:
- Parsing and standardizing inconsistent fields (e.g., dates, phone numbers, emails)
- Deduplicating records and ensuring referential integrity
- Assigning unique identifiers to deals, companies, contacts, and marketing participants
- Structuring the data into clean, relational tables ready for downstream analysis, reporting, or database ingestion
- All functions used are defined in functions.py

---

## **Final Deliverables**

- **`deals_df`**: A clean list of deal opportunities with unique `Deal_IDs`
- **`historical_financial_data_df`**: Historical EBITDA metrics linked by `Deal_ID`
- **`companies_df`**: A master list of companies with unique `Company_IDs`
- **`contacts_df`**: A cleaned list of contacts with unique `Contact_IDs`
- **`marketing_participants_df`**: Event participants with unique `Participant_IDs` linked to `Contact_IDs`

Imports/Installs

In [ ]:
import pandas as pd
import openpyxl
import re
from functions import *

### Business Services Pipeline - Ingestion & Cleaning

In [ ]:
#Ingesting Business Services Pipeline

business_pipeline_df = pd.read_excel("/Users/sonamrupani/Desktop/Intapp Data Engineer Assessment - Data/data_files/Business Services Pipeline.xlsx",
                                     skiprows = 5,
                                     usecols = "A:V")

# leaving 'Date Added' out for additional date parsing/standardization
bsp_dtypes = {
    "Company Name": "string",
    "Project Name": "string",
    "Investment Bank": "string",
    "Banker": "string",
    "Sourcing": "string",
    "Transaction Type": "string",
    "LTM Revenue": "Float64",
    "LTM EBITDA": "Float64",
    "2014A EBITDA": "Float64",
    "2015A EBITDA": "Float64",
    "2016A EBITDA": "Float64",
    "2017A/E EBITDA": "Float64",
    "2018E EBITDA": "Float64",
    "Vertical": "string",
    "Sub Vertical": "string",
    "Enterprise Value": "Float64",
    "Est. Equity Investment": "Float64",
    "Status": "string",
    "Current Owner": "string",
    "Business Description": "string",
    "Lead MD": "string",
    "Notes": "string" 
}

#Financial columns to check
bsp_financial_columns = [
    "LTM Revenue",
    "LTM EBITDA",
    "2014A EBITDA",
    "2015A EBITDA",
    "2016A EBITDA",
    "2017A/E EBITDA",
    "2018E EBITDA",
    "Enterprise Value",
    "Est. Equity Investment"
]

#Cleansing white spaces and new lines
business_pipeline_df = cleanse_column_names(business_pipeline_df)

#Ensure proper null format - pd.NA
business_pipeline_df = modernize_nans(business_pipeline_df)

#Rename columns
business_pipeline_df = business_pipeline_df.rename(columns={"Invest. Bank": "Investment Bank", "Equity Investment Est.": "Est. Equity Investment"})

#TODO double check
#hardcoding CAD to USD rate as 0.73
business_deals, business_deals_audit_df= process_financial_dataframe(business_pipeline_df, bsp_financial_columns, cad_to_usd_rate=0.73)

business_deals = business_deals.astype(bsp_dtypes)

# 1. Backup original Date Added column
business_deals["Date Added (Original)"] = business_deals["Date Added"]

# 2. Apply the simple_date_parser function row-by-row
business_deals["Date Added"] = business_deals["Date Added (Original)"].apply(date_parsing)

display(business_deals)

### Consumer Retail & Healthcare Pipeline - Ingestion & Cleaning

In [ ]:
consumer_retail_healthcare_pipeline = pd.read_excel("Consumer Retail and Healthcare Pipeline.xlsx",\
                                                    skiprows = 8, \
                                                    usecols = "B:W")

#Cleansing white spaces and new lines
consumer_retail_healthcare_pipeline = cleanse_column_names(consumer_retail_healthcare_pipeline)

consumer_retail_healthcare_pipeline = modernize_nans(consumer_retail_healthcare_pipeline)

#Excel formatting causing null trailing rows - remove
consumer_retail_healthcare_pipeline = consumer_retail_healthcare_pipeline.dropna(subset=['Company Name'])
cols = [col for col in consumer_retail_healthcare_pipeline.columns if col != 'Company Name']
consumer_retail_healthcare_pipeline = consumer_retail_healthcare_pipeline[~consumer_retail_healthcare_pipeline[cols].isna().all(axis=1)]

consumer_retail_health_deals = consumer_retail_healthcare_pipeline.copy()

#Leave our dates for further processing
crhp_dtypes = {
    "Company Name": "string",
    "Project Name": "string",
    "Banker": "string",
    "Banker Email": "string",
    "Banker Phone Number": "string",
    "Sourcing": "string",
    "Transaction Type": "string",
    "LTM Revenue": "Float64",
    "LTM EBITDA": "Float64",
    "Vertical": "string",
    "Sub Vertical": "string",
    "Enterprise Value": "Float64",
    "Est. Equity Investment": "Float64",
    "Status": "string",
    "Portfolio Company Status": "string",
    "Active Stage": "string",
    "Passed Rationale": "string",
    "Current Owner": "string",
    "Business Description": "string",
    "Lead MD": "string",
    "Date Added": "datetime64[ns]",
    "Date Added (Original)": "datetime64[ns]",
    "Invest. Bank": "string"
}

chrp_financial_cols = ['LTM Revenue', 'LTM EBITDA', 'Enterprise Value', 'Est. Equity Investment']

consumer_retail_health_deals, crhp_audit_log = process_financial_dataframe(consumer_retail_health_deals, chrp_financial_cols)

# 1. Backup original Date Added column
consumer_retail_health_deals["Date Added (Original)"] = consumer_retail_health_deals["Date Added"]

# 2. Apply the simple_date_parser function row-by-row
consumer_retail_health_deals["Date Added"] = consumer_retail_health_deals["Date Added (Original)"].apply(date_parsing)

consumer_retail_health_deals = update_data_types(consumer_retail_health_deals, crhp_dtypes)

consumer_retail_health_deals = consumer_retail_health_deals.rename(columns = {"Invest. Bank": "Investment Bank"})

display(consumer_retail_health_deals)


### Contacts - Ingest & Clean

In [ ]:
tier_1_contacts = pd.read_excel("Contacts.xlsx", sheet_name = "Tier 1's")
tier_1_contacts["Tier"] = 1

tier_2_contacts = pd.read_excel("Contacts.xlsx", sheet_name = "Tier 2's")
tier_2_contacts["Tier"] = 2

contacts_df = pd.concat([tier_1_contacts, tier_2_contacts], ignore_index = True)

#Cleansing white spaces and new lines
contacts_df = cleanse_column_names(contacts_df)

contacts_df = modernize_nans(contacts_df)

contacts_dtypes = {
    "Firm": "string",
    "Name": "string",
    "Title": "string",
    "Group": "string",
    "Sub-Vertical": "string",
    "E-mail": "string",
    "Phone": "string",
    "Secondary Phone": "string",
    "City": "string",
    "Coverage Person": "string",
    "Preferred Contact Method": "string"
}

contacts_df = update_data_types(contacts_df, contacts_dtypes)

contacts_df['Birthday'] = pd.to_datetime(contacts_df['Birthday'])

display(contacts_df)

### Events - Ingest & Clean

In [ ]:
leaders_partners_events = pd.read_excel("Events.xlsx", sheet_name = "Leaders and Partners Dinner")
leaders_partners_events['Event'] = "Leaders and Partners Dinner"

market_recap = pd.read_excel("Events.xlsx", sheet_name="2019 Market Re-Cap")
market_recap['Event'] = "2019 Market Re-Cap"

events_df = pd.concat([leaders_partners_events, market_recap], ignore_index=True)

#Cleansing white spaces and new lines
events_df = cleanse_column_names(events_df)

events_df = modernize_nans(events_df)

events_dtypes = {
    "Name": "string",
    "E-mail": "string",
    "Attendee Status": "string",
    "Event": "string"
}

events_df = update_data_types(events_df, events_dtypes)

display(events_df)

### PE Comps - Ingest & Clean

In [ ]:
pe_companies = pd.read_excel("PE Comps.xlsx",
                             skiprows = 2,)

#Dropping empty row between header and data
pe_companies = pe_companies.drop(0)

#Cleansing white spaces and new lines
pe_companies = cleanse_column_names(pe_companies)

pe_companies = modernize_nans(pe_companies)

pe_companies['AUM(Mns)'] = pe_companies['AUM(Bns)'] * 1000

pe_companies_dtypes = {
    "Priority": "string", #empty, but keeping string for flexibility
    "Company Name": "string",
    "Website": "string",
    "Sectors": "string",
    "Sample Portfolio Companies": "string",
    "Contact Name 1": "string",
    "Contact 2": "string",
    "Comments": "string",
}

pe_companies = update_data_types(pe_companies, pe_companies_dtypes)

pe_companies = pe_companies.rename(columns = {"Contact 2": "Contact Name 2"})


for col in pe_companies.columns:
    pe_companies[col] = pe_companies[col].apply(clean_dash_text)

pe_companies_df = pe_companies[[
    "Priority", 
    "Company Name", 
    "Website", 
    "AUM(Mns)", 
    "Sectors", 
    "Sample Portfolio Companies", 
    "Contact Name 1",
    "Contact Name 2",
    "Comments"]]

display(pe_companies_df)


In [ ]:
# FINAL INGESTED AND CLEANED DATAFRAMES

display(business_deals)
display(consumer_retail_health_deals)
display(contacts_df)
display(events_df)
display(pe_companies_df)

### Data Modeling - Would make this into another portion of the pipeline

In [ ]:
#Understand cols to ensure proper joins
print(f"Business Deals: {business_deals.columns.tolist()}")
print(f"Consumer Deals: {consumer_retail_health_deals.columns.tolist()}")
print(f"Events DF: {events_df.columns.tolist()}")
print(f"Contacts DF: {contacts_df.columns.tolist()}")
print(f"PE Companies DF: {pe_companies_df.columns.tolist()}")

In [ ]:
# CREATION OF DEALS DATASET

#Removing historical EBITDA metrics, will be added into separate table
business_deals = business_deals.reset_index(drop=True)
business_deals['Deal_ID'] = business_deals.index.map(lambda x: f"D{x+1:04d}")

#Table dedicated to historical financial metrics
historical_financial_data_df = business_deals[[
    "Deal_ID", "Company Name", "Project Name", 
    "2014A EBITDA", "2015A EBITDA", "2016A EBITDA", "2017A/E EBITDA", "2018E EBITDA"
]]

# create Deal_ID before segmenting into deals DF and financial history DF to allow for joins
business_deals_df = business_deals[[
    'Deal_ID', 'Company Name', 'Project Name', 'Date Added', 'Investment Bank', 'Banker',
    'Sourcing', 'Transaction Type', 'LTM Revenue', 'LTM EBITDA', 'Vertical', 'Sub Vertical',
    'Enterprise Value', 'Est. Equity Investment', 'Status', 'Current Owner', 'Business Description',
    'Lead MD', 'Notes'
]].copy()

#Include missing columns from CRHP data. Will remain empty at the moment, but would normally work with client to find a way to populate
business_deals_df['Banker Email'] = pd.Series(pd.NA, dtype='string')
business_deals_df['Banker Phone Number'] = pd.Series(pd.NA, dtype='string')
business_deals_df['Portfolio Company Status'] = pd.Series(pd.NA, dtype='string')
business_deals_df['Active Stage'] = pd.Series(pd.NA, dtype='string')
business_deals_df['Passed Rationale'] = pd.Series(pd.NA, dtype='string')

#Create empty field for proper column references after concatenation
consumer_retail_health_deals['Deal_ID'] = pd.Series(pd.NA, dtype='string')

#Copy to preserve original cleaned dataframe
consumer_deals_df = consumer_retail_health_deals[[
    'Deal_ID', 'Company Name', 'Project Name', 'Date Added', 'Investment Bank', 'Banker', 'Banker Email',
    'Banker Phone Number', 'Sourcing', 'Transaction Type', 'LTM Revenue', 'LTM EBITDA', 'Vertical',
    'Sub Vertical', 'Enterprise Value', 'Est. Equity Investment', 'Status', 'Portfolio Company Status',
    'Active Stage', 'Passed Rationale', 'Current Owner', 'Business Description', 'Lead MD', 'Notes'
]].copy()

#Concatenate the two
deals_df = pd.concat([business_deals_df, consumer_deals_df], ignore_index=True)

deals_df = deals_df.reset_index(drop=True)

# Fill missing Deal_IDs
deals_df['Deal_ID'] = deals_df.apply(
    lambda row: row['Deal_ID'] if pd.notna(row['Deal_ID']) else f"D{row.name+1:04d}",
    axis=1
)

deals_df = modernize_nans(deals_df)

deals_df_reordered = desired_order = [
    'Deal_ID', 'Company Name', 'Project Name', 'Date Added', 'Investment Bank', 'Banker',
    'Sourcing', 'Transaction Type', 'LTM Revenue', 'LTM EBITDA', 'Vertical', 'Sub Vertical',
    'Enterprise Value', 'Est. Equity Investment', 'Status', 'Current Owner', 'Business Description',
    'Lead MD', 'Banker Email', 'Banker Phone Number', 'Portfolio Company Status', 'Active Stage',
    'Passed Rationale', 'Notes'
]

deals_df = deals_df[deals_df_reordered]

deals_df['Banker Phone Number'] = deals_df['Banker Phone Number'].apply(clean_phone)

display(deals_df)

In [ ]:
#COMPANIES DF

pe_companies_subset = pe_companies_df[[
    'Company Name',
    'Website',
    'AUM(Mns)',
    'Sectors',
    'Sample Portfolio Companies',
    'Priority',
    'Comments'
]].copy()
 
deals_companies_subset = deals_df[[
    'Company Name',
    'Business Description',
    'Current Owner'
]].copy()

# Drop duplicates because same Company Name might appear in multiple deals
deals_companies_subset = deals_companies_subset.drop_duplicates(subset='Company Name')

#Normally would do an API call to look for certain fields like business description, current owner, website, sectors, sample portfolio companies
companies_df = pd.merge(
    deals_companies_subset,
    pe_companies_subset,
    how = "outer",
    on = "Company Name"
)

#Clean up nulls
companies_df = modernize_nans(companies_df)

#Clean up after join
companies_df = companies_df.dropna(how = "all")

#Reset index
companies_df = companies_df.reset_index(drop=True)

#Create Company ID
companies_df['Company_ID'] = companies_df.index.map(lambda x: f"CO{x+1:04d}")

#Moving to first column
cols = ['Company_ID'] + [col for col in companies_df.columns if col != 'Company_ID']
companies_df = companies_df[cols]

display(companies_df)


In [ ]:
#Make a copy
contacts_df = contacts_df.copy()

#Strip leading/trailing spaces
contacts_df = contacts_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#Clean Name (Title Case it)
contacts_df['Name'] = contacts_df['Name'].apply(lambda x: x.title() if pd.notna(x) else x)

#Clean Email (lowercase)
contacts_df['Email'] = contacts_df['Email'].apply(lambda x: x.lower().strip() if pd.notna(x) else x)

#Clean Phone numbers (remove non-numeric)
contacts_df['Phone'] = contacts_df['Phone'].apply(clean_phone)
contacts_df['Secondary Phone'] = contacts_df['Secondary Phone'].apply(clean_phone)

#Clean Birthday
contacts_df['Birthday'] = pd.to_datetime(contacts_df['Birthday'], errors='coerce')

#Fill missing values
contacts_df['Preferred Contact Method'] = contacts_df['Preferred Contact Method'].fillna('Email')

#Reset index first
contacts_df = contacts_df.reset_index(drop=True)

#Create 
contacts_df['Contact_ID'] = contacts_df.index.map(lambda x: f"C{x+1:04d}")

#Move Contact_ID to first column for cleanliness
cols = ['Contact_ID'] + [col for col in contacts_df.columns if col != 'Contact_ID']
contacts_df = contacts_df[cols]

display(contacts_df)

In [ ]:
#Create marketing_participants_df
marketing_participants_df = events_df.rename(columns={
    'Name': 'Contact Name',
    'E-mail': 'Email',
    'Event': 'Event Name'
}).copy()

#Reset index
marketing_participants_df = marketing_participants_df.reset_index(drop=True)

#Create Participant_ID
marketing_participants_df['Participant_ID'] = marketing_participants_df.index.map(lambda x: f"M{x+1:04d}")

#Move ID to first column for cleanliness
cols = ['Participant_ID'] + [col for col in marketing_participants_df.columns if col != 'Participant_ID']
marketing_participants_df = marketing_participants_df[cols]

#Lowercase and strip emails
contacts_df['Email'] = contacts_df['Email'].str.lower().str.strip()
marketing_participants_df['Email'] = marketing_participants_df['Email'].str.lower().str.strip()

columns_to_drop = [col for col in marketing_participants_df.columns if 'Contact_ID' in col]
if columns_to_drop:
    marketing_participants_df = marketing_participants_df.drop(columns=columns_to_drop)

#Merge Contact_ID onto marketing_participants_df - grab contact ID
marketing_participants_df = marketing_participants_df.merge(
    contacts_df[['Contact_ID', 'Email']],
    on='Email',
    how='left'
)

#Reorder columns cleanly
cols = ['Participant_ID', 'Contact_ID'] + [col for col in marketing_participants_df.columns if col not in ['Participant_ID', 'Contact_ID']]
marketing_participants_df = marketing_participants_df[cols]

#Reset index
marketing_participants_df = marketing_participants_df.reset_index(drop=True)

display(marketing_participants_df)

In [ ]:
# Save final outputs
deals_df.to_excel('final_deals.xlsx', index=False)
historical_financial_data_df.to_excel('final_financial_data.xlsx', index=False)
companies_df.to_excel('final_companies.xlsx', index=False)
contacts_df.to_excel('final_contacts.xlsx', index=False)
marketing_participants_df.to_excel('final_marketing_participants.xlsx', index=False)

print("All files saved successfully!")